# 単純な量子回路をゼロから書く

{doc}`第一回の前半 <chsh_inequality>`で、量子回路の基本的な書き方と実機での実行の仕方を学びました。そこで出てきたように、QCでは量子レジスタに特定の量子状態を実現することが計算を行うこととなります。

後半では、基本的なゲートを組み合わせて様々な量子状態を作る実習を行います。

```{contents} 目次
---
local: true
---
```

$\newcommand{ket}[1]{|#1\rangle}$

## シミュレータで量子状態を調べる

前半で見たように、QCで実現される量子状態は、量子力学の公理に基づいて理論的に計算・予測できます。そこで用いられる数学的操作も単なる足し算や掛け算（線形演算）なので、実はQCの量子状態は（古典）計算機で比較的簡単に計算できます。当然のことですが、QCは何も魔法のブラックボックスというわけではありません。

ただし、古典計算機で量子状態を再現するためには、特殊な場合を除いて、量子ビット数の指数関数的な量のメモリが必要になります。これも前半で見たように、$n$量子ビットあれば、系の自由度（degrees of freedom / d.o.f.: 実数自由パラメータの数）は$2^{n+1} - 2$ですので、例えば各自由度を64ビットの浮動小数点で表現するとしたら、必要なメモリは(-2を無視して)

$$
2^6\, \mathrm{(bits / d.o.f.)} \times 2^{n+1}\, \mathrm{(d.o.f.)} \times 2^{-3}\, \mathrm{(bytes / bit)} = 2^{n+4}\, \mathrm{(bytes)}
$$

なので、$n=16$で1 MiB、$n=26$で1 GiB、$n=36$で1 TiBです。現在の計算機では、よほどのハイエンドワークステーションでもRAMは$\mathcal{O}(1)$ TiBなのに対し、QCではすでに$n>50$のものが存在するので、後者が使いこなせて有用な問題を解くことができればいわゆる「量子超越」(quantum supremacy)が達成されたということになります。

話が逸れましたが、逆に言うと、$n \sim 30$程度までの回路であれば、ある程度のスペックを持った計算機で厳密にシミュレートできるということが言えます。じっさい世の中には[数多くの](https://quantiki.org/wiki/list-qc-simulators)シミュレータが存在します。

実機での計算に対するシミュレーションの利点として、

- **量子振幅が直接見える**：実機では回路を繰り返し実行・測定することで計算基底に対する確率分布（＝量子振幅の絶対値自乗）を推定することができますが、複素位相までを含めた振幅の情報を知ることはできません。
- **回路の途中の状態を調べられる**：実機でも回路を測定で中断することでその時点での状態をある程度調べることができますが、専用の回路を用意しなければいけません。
- **エラーがない**：前半の実習の最後で見たように、NISQの実機には様々なエラーがあり、計算結果が予測と大きくずれることがあります。シミュレータではエラーの影響を受けずに純粋に回路の状態を調べることができます。

などが挙げられます。

Qiskitにも様々な高機能シミュレータが同梱されているので、その使い方と出力の読み方を説明します。

まずは`Aer`というオブジェクトをインポートします。

In [1]:
from qiskit import Aer

`Aer`は前半に登場した`IBMQ`と同様の構造をしており、複数のシミュレータをバックエンドとして管理しています。量子振幅を直接観察するには、そのうちの`statevector_simulator`を使用します。

In [2]:
simulator = Aer.get_backend('statevector_simulator')

シミュレートする回路を作ります。ここでは例として前半で登場した回路1を使います。測定以外のゲートをかけていきます。

In [9]:
from qiskit import QuantumCircuit
import numpy as np

circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.ry(3. * np.pi / 4., 1)

```{warning}
回路に測定操作(`measure_all()`)を加えてしまうと、`statevector_simulator`を利用して測定前の回路の量子状態を確認することができなくなります。
```

シミュレーションを実行する際も、実機のバックエンドと同様に`execute`関数を使い、ジョブオブジェクトから結果を得ます。ただし、今回は計算結果からカウントではなく「状態ベクトル」（＝量子振幅の配列）を得るので`get_counts`ではなく`result.data()['statevector']`を参照します。

In [10]:
from qiskit import execute
job = execute(circuit, simulator)
result = job.result()
statevector = result.data()['statevector']

print(type(statevector), statevector.dtype)
print(statevector)

<class 'numpy.ndarray'> complex128
[ 0.27059805+0.j -0.65328148+0.j  0.65328148+0.j  0.27059805+0.j]


状態ベクトルは[numpy](https://numpy.org/)の`ndarray`オブジェクトとして実装されています。データタイプは128ビットの複素数（実部と虚部それぞれ64ビット）です。アレイのインデックスがそのまま二進数としてみなした計算基底の値に対応しています。

In [21]:
from IPython.display import display, Markdown

state_string = r'$\ket{\psi} ='
for idx in range(statevector.shape[0]):
    angle = np.arcsin(statevector[idx].real * np.sqrt(2.))
    norm_angle = np.round(angle / np.pi * 8.).astype(int)
    state_string += r' \sin({} \pi / 8)\ket{{{}}}'.format('' if norm_angle == 1 else norm_angle, idx)
    if idx != statevector.shape[0] - 1:
        state_string += '+'

state_string += r'$'
display(Markdown(state_string))

$\ket{\psi} = \sin( \pi / 8)\ket{0}+ \sin(-3 \pi / 8)\ket{1}+ \sin(3 \pi / 8)\ket{2}+ \sin( \pi / 8)\ket{3}$

## その他のゲート

前半でよく使われる1量子ビットゲートと2量子ビット制御ゲートを紹介しましたが、他にも知っておくと便利なゲートがいくつかあります。

### 1量子ビット

- $U_1$または$P$: パラメータ$\phi$を取り、$\ket{1}$に位相$e^{i\phi}$をかける。$P(\phi)$は$R_{z}(\phi)$と等価なので、1量子ビットゲートとして利用する分にはどちらでも同じ結果が得られます。ただし2量子ビット制御ゲートとしては$C[R_z(\phi)]$と$C[P(\phi)]$は異なる働きをします。
- $U_3$: パラメータ$\theta, \phi, \lambda$を取り、単一量子ビットの任意の状態を実現する。
    ```{math}
    U_3(\theta, \phi, \lambda)\ket{0} = \cos\left(\frac{\theta}{2}\right) \ket{0} + e^{i\phi}\sin\left(\frac{\theta}{2}\right) \ket{1} \\
    U_3(\theta, \phi, \lambda)\ket{1} = -e^{i\lambda}\sin\left(\frac{\theta}{2}\right) \ket{0} + e^{i(\phi+\lambda)}\cos\left(\frac{\theta}{2}\right) \ket{1}
    ```

### 2量子ビット

- SWAP: 二つの量子ビットの量子状態を交換する。CNOT三回で実装できる。
    ```{math}
    \begin{align}
    & C^0_1[X]C^1_0[X]C^0_1[X] (\alpha \ket{00} + \beta \ket{01} + \gamma \ket{10} + \delta \ket{11}) \\
    = & C^0_1[X]C^1_0[X] (\alpha \ket{00} + \beta \ket{01} + \gamma \ket{11} + \delta \ket{10}) \\
    = & C^0_1[X] (\alpha \ket{00} + \beta \ket{11} + \gamma \ket{01} + \delta \ket{10}) \\
    = & \alpha \ket{00} + \beta \ket{10} + \gamma \ket{01} + \delta \ket{11}
    \end{align}
    ```

### 3量子ビット以上

量子レジスタに対する全ての操作が1量子ビットゲートと2量子ビット制御ゲート（実際にはCNOTのみ）の組み合わせで表現できることはすでに触れましたが、実際に量子回路を書くときには3つ以上の量子ビットにかかるゲートを使用したほうがアルゴリズムをコンパクトに表現できます。そのようなゲートを含む回路を実機で実行するときには、「トランスパイル」（transpilation）という過程を経て最終的には全て1量子ビットゲートとCNOTに自動的に分解されます。

3量子ビット以上にかかるゲートも基本は制御ゲートで、よく使われるのは制御ビットが複数あるような多制御（multi-controlled）ゲート$C^{ij\dots}_{k}[U]$です。そのうち制御ビットが2つで標的ビット1つに$X$をかける$C^{ij}_{k}[X]$は特別多用され、Toffoliゲートとも呼ばれます。

多制御ゲートでない3量子ビットゲートの例として、controlled SWAPというものも存在します。制御ビット1つで他の2つの量子ビットの間の量子状態交換を制御します。当然制御ビットを増やしたmulti-controlled SWAPというものも考えられます。

## 量子状態生成

## [課題]ダブルスリット実験のシミュレーション